**_Silver Data _Transformation__**

In [0]:
df = spark.read.format("delta")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@datalakenetflixasp.dfs.core.windows.net/netflix_titles")

In [0]:
display(df)

**_Fixing the Null values_**


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons": 1})

In [0]:
df.display()

convert the data types

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))

In [0]:
display(df)

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
     .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
display(df)

In [0]:
df = df.withColumn("ShortTtile", split(col('title'), ':')[0])

In [0]:
display(df)

In [0]:
df = df.withColumn("ShortRating", split(col('rating'), '-')[0])

In [0]:
display(df)

In [0]:
##df = df.withColumn("type_flag", when(col("type")== "Movie"),1).otherwise(0)) - conditional statements

df = df.withColumn("type_flag",when(col("type")== "Movie",1)\
                            .when(col("type")== "TV Show",2)\
                            .otherwise(0))

In [0]:
display(df)

__rank the data based on duration : we can use the sort function : also we can use the rank function from window **functions_**_

In [0]:
## rank the duration_minutes column we can also use the sort but we will use the rank function

from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
display(df)

**_data aggregations_**

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))

In [0]:
display (df_vis)

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path", "abfss://silver@datalakenetflixasp.dfs.core.windows.net/netflix_titles")\
    .save()


In [0]:
df.display